In [ ]:
## Author- Sayan Chandra
## Roll - CS20M057
## Instructor - Mitesh M. Khapra
## Course - CS6910 (Fundamentals of Deep Learning)

In [2]:
## importing all necessary modules
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import torch
import torch.nn as cnn
import torch.optim as optimisations
from torch.nn import functional as func
from torch.utils.data import DataLoader as dataloader
import torchvision.transforms as transforms
import torchvision
from torch.autograd import Variable

In [ ]:
!pip install split-folders
import splitfolders as sf

In [ ]:
!unzip '/content/drive/MyDrive/nature_12K.zip'

In [ ]:
MAINPATH = "/content/inaturalist_12K/"
sf.fixed(MAINPATH + "train", output=MAINPATH+"train_split", seed=1337, fixed=100, oversample=False, group_prefix=None)

Copying files: 9999 files [01:01, 161.53 files/s]


In [ ]:
sizew=300; sizeh=300
data_transforms = transforms.Compose([
        transforms.Resize((sizew, sizeh)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [3]:
BATCH_SIZE=16
EPOCHS=20
TESTPATH="/content/inaturalist_12K/val"
def loadTrain_Val_TestData(): # no arguments
  train_data = torchvision.datasets.ImageFolder(root=MAINPATH+"train_split/train/", transform=data_transforms)
  val_data = torchvision.datasets.ImageFolder(root=MAINPATH+"train_split/val/", transform=data_transforms)
  train_data_loader = dataloader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=1)
  val_data_loader = dataloader(val_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=1)
  test_data = torchvision.datasets.ImageFolder(root=TESTPATH, transform=data_transforms)
  test_data_loader  = dataloader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=1)
  return train_data, test_data, val_data, train_data_loader, test_data_loader, val_data_loader


In [38]:
numOfConvPoolLayers=5
sizew=300; sizeh=300
cnn_config=[[3, 128, 11, 1, (0, 0), 3, 2],
            [128, 128, 7, 2, (0, 0), 3, 2],
            [128, 128, 5, 2, (0, 0), 2, 1],
            [128, 128, 5, 1, (0, 0), 2, 1],
            [128, 128, 3, 1, (0, 0), 2, 1],
            "relu"] ## in, out, kernel, stride, pad, actv, maxpoolkernel, maxpoolstride
denseLayer=128
def computeFirstDenseLayer():
  w,h,d=sizew,sizeh,3
  for i in range( numOfConvPoolLayers):
    d=cnn_config[i][1]
    w=1+np.floor((w-cnn_config[i][2]+2* cnn_config[i][4][0])/cnn_config[i][3])
    h=w#1+np.floor((h-cnn_config[i][2]+2* cnn_config[i][4][0])/cnn_config[i][3])
    w=1+np.floor((w-cnn_config[i][5])/cnn_config[i][6])
    h=w#1+np.floor((h-cnn_config[i][5])/cnn_config[i][6])
    print(w,h)
  return w*h*d
print(computeFirstDenseLayer())

144.0 144.0
34.0 34.0
14.0 14.0
9.0 9.0
6.0 6.0
4608.0


In [ ]:
class CNN(cnn.Module):
  def __init__(self, cnn_config, in_channels=3, num_classes=10, denseLayer=denseLayer):
    super(CNN, self).__init__()

    self.conv1=cnn.Conv2d(in_channels=cnn_config[0][0], out_channels=cnn_config[0][1], kernel_size= cnn_config[0][2], stride= cnn_config[0][3], padding= cnn_config[0][4])
    cnn.init.xavier_uniform_(self.conv1.weight)
    self.activ=self.Activ2d(cnn_config[numOfConvPoolLayers])
    self.batchnorm1=cnn.BatchNorm2d(cnn_config[0][1])
    self.maxpool1=cnn.MaxPool2d(cnn_config[0][5], stride=cnn_config[0][6])

    self.conv2=cnn.Conv2d(in_channels=cnn_config[1][0], out_channels=cnn_config[1][1], kernel_size= cnn_config[1][2], stride= cnn_config[1][3], padding= cnn_config[1][4])
    cnn.init.xavier_uniform_(self.conv2.weight)
    self.batchnorm2=cnn.BatchNorm2d(cnn_config[1][1])
    self.maxpool2=cnn.MaxPool2d(cnn_config[1][5], stride=cnn_config[1][6])
    
    self.conv3=cnn.Conv2d(in_channels=cnn_config[2][0], out_channels=cnn_config[2][1], kernel_size= cnn_config[2][2], stride= cnn_config[2][3], padding= cnn_config[2][4])
    cnn.init.xavier_uniform_(self.conv3.weight)
    self.batchnorm3=cnn.BatchNorm2d(cnn_config[2][1])
    self.maxpool3=cnn.MaxPool2d(cnn_config[2][5], stride=cnn_config[2][6])

    self.conv4=cnn.Conv2d(in_channels=cnn_config[3][0], out_channels=cnn_config[3][1], kernel_size= cnn_config[3][2], stride= cnn_config[3][3], padding= cnn_config[3][4])
    cnn.init.xavier_uniform_(self.conv4.weight)
    self.batchnorm4=cnn.BatchNorm2d(cnn_config[3][1])
    self.maxpool4=cnn.MaxPool2d(cnn_config[3][5], stride=cnn_config[3][6])

    self.conv5=cnn.Conv2d(in_channels=cnn_config[4][0], out_channels=cnn_config[4][1], kernel_size= cnn_config[4][2], stride= cnn_config[4][3], padding= cnn_config[4][4])
    cnn.init.xavier_uniform_(self.conv5.weight)
    self.batchnorm5=cnn.BatchNorm2d(cnn_config[4][1])
    self.maxpool5=cnn.MaxPool2d(cnn_config[4][5], stride=cnn_config[4][6])
    
    self.val=computeFirstDenseLayer().astype(int)
    self.fullconn1=cnn.Linear(in_features=self.val, out_features= denseLayer)
    cnn.init.xavier_uniform_(self.fullconn1.weight)
    self.dropout=cnn.Dropout(p=0.5)
    self.output=cnn.Linear(in_features= denseLayer, out_features= num_classes)

  def forward(self, curinp, h):
    print(h)
    out=self.maxpool1(self.batchnorm1(self.activ(self.conv1(curinp))))
    out=self.maxpool2(self.batchnorm2(self.activ(self.conv2(out))))
    out=self.maxpool3(self.batchnorm3(self.activ(self.conv3(out))))
    out=self.maxpool4(self.batchnorm4(self.activ(self.conv4(out))))
    out=self.maxpool5(self.activ(self.conv5(out))) #self.batchnorm5(

    out=out.view(-1, self.val)
    out=self.activ(self.fullconn1(out))
    out=self.dropout(out)
    out=func.softmax(self.output(out), dim=1)

    return out

  def Activ2d(self, str):
    if str=="relu" : return func.relu
    elif str=="relu6" : return func.relu6
    elif str=="rrelu" : return func.rrelu
    elif str=="lrelu" : return func.leaky_relu
    elif str=="silu" : return func.silu
    elif str=="erelu" : return func.elu
    elif str=="softmax" : return func.softmax
    elif str=="tanh" : return func.tanh


In [ ]:
train_data, test_data, val_data, train_data_loader, test_data_loader, val_data_loader = loadTrain_Val_TestData()
cnnModel=CNN(cnn_config)
heyGPU=torch.cuda.is_available()
if heyGPU: cnnModel=cnnModel.cuda()
lossfunc=cnn.CrossEntropyLoss()
#optimizer=optimisations.Adam(cnnModel.parameters(), lr=0.006)
#optimizer=optimisations.AdamW(cnnModel.parameters(), lr=0.001, weight_decay=0.005)
optimizer = torch.optim.SGD(cnnModel.parameters(), lr=1e-3, momentum=0.92)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=int(np.ceil(len(train_data_loader)/BATCH_SIZE)))

147.0 147.0
70.0 70.0
65.0 65.0
30.0 30.0
14.0 14.0


562.4375

In [ ]:
def trainMyModel(EPOCHS, train_data_loader, heyGPU, optimizer, scheduler, val_data_loader):
    itr=0
    for run in range(EPOCHS):
      globalloss=0
      for i, (curimg, ytrue) in enumerate(train_data_loader):
        itr+=1
        if heyGPU : curimg, ytrue=Variable(curimg.cuda()), Variable(ytrue.cuda())
        else : curimg, ytrue=Variable(curimg), Variable(ytrue)
        #print("yoo")
        optimizer.zero_grad()
        outputt=cnnModel(curimg, 789)
        curloss=lossfunc(outputt, ytrue)
        globalloss+=curloss.item()
        curloss.backward()
        optimizer.step()
        scheduler.step()
        #print("yooy")

        
      predictedright=0
      totalimgs=0
      for img, y in val_data_loader:
          if heyGPU : img, y=Variable(img.cuda()), Variable(y.cuda())
          else : img, y=Variable(img), Variable(y)
          outp=cnnModel(img)
          ignore, predicted = torch.max(outp.data, 1)
          totalimgs+=y.size(0)
          if heyGPU : predictedright+=(predicted.cpu()==y.cpu()).sum()
          else : predictedright+=(predicted==y).sum()
      curaccuracy=(predictedright/totalimgs)*100
      print(itr, globalloss/len(train_data), curaccuracy.item()) 
trainMyModel(EPOCHS, train_data_loader, heyGPU, optimizer, scheduler, val_data_loader)

789
789


KeyboardInterrupt: ignored